## sparkinactionhw04
1）以HDFS中文件为输入，做word count操作，观察各Stage的Data locality，并截图。  
2）将HDFS中文件读为RDD，转化为Java集合，并通过broadcast广播到各个executor。通过parallelize方法，创建一个只有一个partition的RDD，对该RDD使用flatMap方法，在flatMap方法中读取广播变量（Java集合），然后作word count操作，观察此时各stage的data locality，并截图  
3）按照视频的步骤，验证Stage间的串行执行，及Stage内的并行执行，以及Task内的pipeline执行。截图证明  

### 1）以HDFS中文件为输入，做word count操作，观察各Stage的Data locality，并截图。  

In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
#生成SparkSession实例
spark = SparkSession.builder \
     .master("local[*]") \
     .appName("Word Count") \
     .config("spark.some.config.option", "some-value") \
     .getOrCreate()

In [22]:
#通过sparkSession获取上下文
sc = spark.sparkContext

In [7]:
rdd1 = sc.textFile("/test.py")

In [8]:
result = rdd1.flatMap(lambda x:x.split()).map(lambda x:(x,1)).reduceByKey(lambda a,b: a+b).collect()

In [9]:
result[:5]

[('paramMap.update({lr.regParam:', 1),
 ('#', 20),
 ('Specify', 2),
 ('multiple', 1),
 ('Params.', 1)]

共有两个stage  
![image.png](https://upload-images.jianshu.io/upload_images/2850424-ad7685f38b768215.png?imageMogr2/auto-orient/strip%7CimageView2/2/w/1240)

第一个stage的data-locality  
![image.png](https://upload-images.jianshu.io/upload_images/2850424-9ff04ac76228c705.png?imageMogr2/auto-orient/strip%7CimageView2/2/w/1240)
第二个stage的data-locality  
![image.png](https://upload-images.jianshu.io/upload_images/2850424-882bb983e2ee2440.png?imageMogr2/auto-orient/strip%7CimageView2/2/w/1240)


### 2）将HDFS中文件读为RDD，转化为Java集合，并通过broadcast广播到各个executor。通过parallelize方法，创建一个只有一个partition的RDD，对该RDD使用flatMap方法，在flatMap方法中读取广播变量（Java集合），然后作word count操作，观察此时各stage的data locality，并截图 

In [1]:
#通过sparkSession获取上下文
sc = spark.sparkContext

In [2]:
rdd1 = sc.textFile("/test.py")

In [3]:
broadcastValues = sc.broadcast(rdd1.collect())

In [4]:
parti = sc.parallelize(broadcastValues.value,1)

In [5]:
r1 = parti.flatMap(lambda x:x.split()).map(lambda x:(x,1)).reduceByKey(lambda a,b: a+b).collect()
r1[:5]

[('from', 4),
 ('pyspark.ml.linalg', 1),
 ('import', 3),
 ('Vectors', 1),
 ('pyspark.ml.classification', 1)]

第一个stage的data-locality  
![image.png](https://upload-images.jianshu.io/upload_images/2850424-4589e3ccbedb2c1c.png?imageMogr2/auto-orient/strip%7CimageView2/2/w/1240)  
第二个stage的data-locality  
![image.png](https://upload-images.jianshu.io/upload_images/2850424-40215f0b375db680.png?imageMogr2/auto-orient/strip%7CimageView2/2/w/1240)

### 3）按照视频的步骤，验证Stage间的串行执行，及Stage内的并行执行，以及Task内的pipeline执行。截图证明 

认真的看了老师视频中的操作，深刻理解了“Stage间的串行执行，及Stage内的并行执行，以及Task内的pipeline执行”的运行过程。  
但是由于我用的是python，所用IDE无法进行单步调试，故无法进行这个过程。望见谅！